# The CHSH game

In [31]:
from qiskit import QuantumCircuit
from qiskit_aer.primitives import Sampler
from qiskit.visualization import plot_histogram

from numpy import pi 
from numpy.random import randint

In [32]:
def chsh_circuit(x, y):
    """
    Creates a `QuantumCircuit` that implements the best CHSH strategy.
    Args:
        x (int): Alice's bit (must be 0 or 1)
        y (int): Bob's bit (must be 0 or 1)
    Returns:
        QuantumCircuit: Circuit that, when run, returns Alice and Bob's
            answer bits.
    """

    circuit = QuantumCircuit(2, 2)

    circuit.h(0)
    circuit.cx(0, 1)

    circuit.barrier()

    if x == 0:
        circuit.ry(0, 0)
    else:
        circuit.ry(-pi / 2, 0)

    circuit.measure(0, 0)

    if y == 0:
        circuit.ry(-pi / 4, 1)
    else:
        circuit.ry(pi / 4, 1)

    circuit.measure(1, 1)

    return circuit

In [33]:
print("(x, y) = (0, 0)")
display(chsh_circuit(0, 0).draw())

print("(x, y) = (0, 1)")
display(chsh_circuit(0, 1).draw())

print("(x, y) = (1, 0)")
display(chsh_circuit(1, 0).draw())

print("(x, y) = (1, 1)")
display(chsh_circuit(1, 1).draw())

(x, y) = (0, 0)


┌───┐      ░  ┌───────┐  ┌─┐   
q_0: ┤ H ├──■───░──┤ Ry(0) ├──┤M├───
     └───┘┌─┴─┐ ░ ┌┴───────┴─┐└╥┘┌─┐
q_1: ─────┤ X ├─░─┤ Ry(-π/4) ├─╫─┤M├
          └───┘ ░ └──────────┘ ║ └╥┘
c: 2/══════════════════════════╩══╩═
                               0  1

(x, y) = (0, 1)


┌───┐      ░  ┌───────┐ ┌─┐   
q_0: ┤ H ├──■───░──┤ Ry(0) ├─┤M├───
     └───┘┌─┴─┐ ░ ┌┴───────┴┐└╥┘┌─┐
q_1: ─────┤ X ├─░─┤ Ry(π/4) ├─╫─┤M├
          └───┘ ░ └─────────┘ ║ └╥┘
c: 2/═════════════════════════╩══╩═
                              0  1

(x, y) = (1, 0)


┌───┐      ░ ┌──────────┐┌─┐   
q_0: ┤ H ├──■───░─┤ Ry(-π/2) ├┤M├───
     └───┘┌─┴─┐ ░ ├──────────┤└╥┘┌─┐
q_1: ─────┤ X ├─░─┤ Ry(-π/4) ├─╫─┤M├
          └───┘ ░ └──────────┘ ║ └╥┘
c: 2/══════════════════════════╩══╩═
                               0  1

(x, y) = (1, 1)


┌───┐      ░ ┌──────────┐┌─┐   
q_0: ┤ H ├──■───░─┤ Ry(-π/2) ├┤M├───
     └───┘┌─┴─┐ ░ ├─────────┬┘└╥┘┌─┐
q_1: ─────┤ X ├─░─┤ Ry(π/4) ├──╫─┤M├
          └───┘ ░ └─────────┘  ║ └╥┘
c: 2/══════════════════════════╩══╩═
                               0  1

In [34]:
sample = Sampler()


def chsh_strategy(x, y):
    """
    Carry out the best strategy for the CHSH game.
    Args:
        x (int): Alice's bit (must be 0 or 1)
        y (int): Bob's bit (must be 0 or 1)
    Returns:
        (int, int): Alice and Bob's answer bits (respectively)
    """

    result = sample.run(chsh_circuit(x, y), shots=1).result()

    binary_probabilities = result.quasi_dists[0].binary_probabilities()

    bits = list(binary_probabilities.keys())[0]

    a, b = bits[0], bits[1]

    return a, b

In [35]:
def chsh_game(chsh_strategy):
    """
    Plays the CHSH game
    Args:
        strategy (callable): A function that takes two bits (as `int`s) and
            returns two bits (also as `int`s). The strategy must follow the
            rules of the CHSH game.
    Returns:
        int: 1 for a win, 0 for a loss.
    """

    x, y = randint(0, 2), randint(0, 2)

    a, b = chsh_strategy(x, y)

    return 1 if (a != b) == (x & y) else 0

In [36]:
COUNT = 1_000
SCORE = 0

for _ in range(COUNT):
    SCORE += chsh_game(chsh_strategy)

print("Fraction of games won: ", SCORE / COUNT)

Fraction of games won:  0.85
